<a href="https://colab.research.google.com/github/GonkBunny/AIFaceRecognition/blob/main/BioActivity_ML_data_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1 - Computational Drug Discovery**
This follows the https://www.youtube.com/watch?v=jBlTQjcKuaY&t=1057s

## **Data collection**

### **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.
[Data as of March 25, 2020; ChEMBL version 26].

### **Installing Libraries**

The first step is to install the ChEMBL web service package so we can retrieve every bioactivity from the ChEMBL Database

In [29]:
pip install chembl_webresource_client

In [30]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target Protein**

### **Target search for Coronavirus**

In [31]:
target = new_client.target
target_query = target.search('coronavirus')
target_query

[{'cross_references': [], 'organism': 'Coronavirus', 'pref_name': 'Coronavirus', 'score': 17.0, 'species_group_flag': False, 'target_chembl_id': 'CHEMBL613732', 'target_components': [], 'target_type': 'ORGANISM', 'tax_id': 11119}, {'cross_references': [], 'organism': 'SARS coronavirus', 'pref_name': 'SARS coronavirus', 'score': 15.0, 'species_group_flag': False, 'target_chembl_id': 'CHEMBL612575', 'target_components': [], 'target_type': 'ORGANISM', 'tax_id': 227859}, {'cross_references': [], 'organism': 'Feline coronavirus', 'pref_name': 'Feline coronavirus', 'score': 15.0, 'species_group_flag': False, 'target_chembl_id': 'CHEMBL612744', 'target_components': [], 'target_type': 'ORGANISM', 'tax_id': 12663}, {'cross_references': [], 'organism': 'Human coronavirus 229E', 'pref_name': 'Human coronavirus 229E', 'score': 13.0, 'species_group_flag': False, 'target_chembl_id': 'CHEMBL613837', 'target_components': [], 'target_type': 'ORGANISM', 'tax_id': 11137}, '...(remaining elements truncate

Since it's a dict, convert to a DataFrame using pandas

In [32]:
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Coronavirus,Coronavirus,17.0,False,CHEMBL613732,[],ORGANISM,11119
1,[],SARS coronavirus,SARS coronavirus,15.0,False,CHEMBL612575,[],ORGANISM,227859
2,[],Feline coronavirus,Feline coronavirus,15.0,False,CHEMBL612744,[],ORGANISM,12663
3,[],Human coronavirus 229E,Human coronavirus 229E,13.0,False,CHEMBL613837,[],ORGANISM,11137
4,"[{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...",SARS coronavirus,SARS coronavirus 3C-like proteinase,10.0,False,CHEMBL3927,"[{'accession': 'P0C6U8', 'component_descriptio...",SINGLE PROTEIN,227859
5,[],Middle East respiratory syndrome-related coron...,Middle East respiratory syndrome-related coron...,9.0,False,CHEMBL4296578,[],ORGANISM,1335626
6,"[{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...",SARS coronavirus,Replicase polyprotein 1ab,4.0,False,CHEMBL5118,"[{'accession': 'P0C6X7', 'component_descriptio...",SINGLE PROTEIN,227859
7,[],Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,4.0,False,CHEMBL4523582,"[{'accession': 'P0DTD1', 'component_descriptio...",SINGLE PROTEIN,2697049


### **Select and retrieve bioactivity data for SARS coronavirus 2 Replicase polyprotein 1ab(eighth entry)**

We will assign the eighth entry (which corresponds to the target protein, coronavirus 2) to the selected_target variable

In [33]:
selected_target = targets.target_chembl_id[7]
selected_target

'CHEMBL4523582'

Here, we will retrieve only bioactivity data for coronavirus polyprotein (CHEMBL4523582) that are reported as IC 50  values in nM (nanomolar) unit.

In [34]:
activity = new_client.activity
res = activity.filter( target_chembl_id=selected_target).filter(standard_type="IC50")

In [35]:
df = pd.DataFrame.from_dict(res)

In [36]:
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,Dtt Insensitive,19964199,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,Cc1c(OCC(F)(F)F)ccnc1C[S+]([O-])c1nc2ccccc2[nH]1,None,None,CHEMBL4495564,None,2020,None,CHEMBL480,LANSOPRAZOLE,CHEMBL480,6.41,False,http://www.openphacts.org/units/Nanomolar,3341963,=,52,True,=,None,IC50,nM,None,390.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,0.39
1,Dtt Insensitive,19964200,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,Cc1c(-c2cnccn2)ssc1=S,None,None,CHEMBL4495564,None,2020,None,CHEMBL178459,OLTIPRAZ,CHEMBL178459,6.68,False,http://www.openphacts.org/units/Nanomolar,3341991,=,52,True,=,None,IC50,nM,None,210.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,0.21
2,Dtt Insensitive,19964201,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,O=c1sn(-c2cccc3ccccc23)c(=O)n1Cc1ccccc1,None,None,CHEMBL4495564,None,2020,None,CHEMBL3545157,TIDEGLUSIB,CHEMBL3545157,7.10,False,http://www.openphacts.org/units/Nanomolar,3342067,=,52,True,=,None,IC50,nM,None,80.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,0.08
3,Dtt Insensitive,19964202,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(O)...,None,None,CHEMBL4495564,None,2020,None,CHEMBL297453,EPIGALOCATECHIN GALLATE,CHEMBL297453,5.80,False,http://www.openphacts.org/units/Nanomolar,3342156,=,52,True,=,None,IC50,nM,None,1580.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,1.58
4,Dtt Insensitive,19964203,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,O=C1C=Cc2cc(Br)ccc2C1=O,None,None,CHEMBL4495564,None,2020,None,CHEMBL4303595,None,CHEMBL4303595,7.40,False,http://www.openphacts.org/units/Nanomolar,3342307,=,52,True,=,None,IC50,nM,None,40.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Dtt Insensitive,19964311,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,C=CC(=O)c1ccc2ccccc2c1,None,None,CHEMBL4495564,None,2020,None,CHEMBL154580,None,CHEMBL154580,5.91,False,http://www.openphacts.org/units/Nanomolar,3350392,=,52,True,=,None,IC50,nM,None,1240.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,1.24
113,Dtt Insensitive,19964312,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,C[n+]1c2cc(N)ccc2cc2ccc(N)cc21.[Cl-],None,None,CHEMBL4495564,None,2020,None,C

In [37]:
df.standard_type.unique()

array(['IC50'], dtype=object)

Saving the resulting bioactivity data to a CSV file **bioactivity_data.csv**

In [38]:
df.to_csv('bioactivity_data.csv', index=False)

### Copying files to Google Drive

In [39]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount= True)


Mounted at /content/gdrive/


In [40]:
! ls "/content/gdrive/My Drive/"

'1538_001 (1).pdf'	        giphy.gif
 Backup			        GoncaloCoelho152757708ZX0.jpg
'Colab Notebooks'	        Ideias.gdoc
 Curiosidades.gdoc	        Partituras
 CV-Goncalo_Coelho-PT.pdf      'PGI-CheapShop!'
 Dataset		        phrasal_verbs_list.pdf
'Documento sem nome (1).gdoc'   Relatorio.gdoc
'Documento sem nome.gdoc'       SD_Meta2.gdoc
'EA - Ultimo Relatorio.gdoc'    Survey.gdoc
'Excell Perguntas.gsheet'       Video
 Experiencias.gsheet	        VotaçãoParaEnsaiarBandolim.gsheet


In [41]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/"
! mkdir "/content/gdrive/My Drive/Colab Notebooks/data"

mkdir: cannot create directory ‘/content/gdrive/My Drive/Colab Notebooks/’: File exists
mkdir: cannot create directory ‘/content/gdrive/My Drive/Colab Notebooks/data’: File exists


In [42]:
! cp bioactivity_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

In [43]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/data"

total 68
-rw------- 1 root root 58735 Aug  9 20:35 bioactivity_data.csv
-rw------- 1 root root 10578 Aug  9 20:33 bioactivity_preprocessed_data.csv


## **Handling missing data**

If any compound has missing value for **standard_value** column then drop it

In [44]:
df2 = df[df.standard_value.notna()]
df2

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,Dtt Insensitive,19964199,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,Cc1c(OCC(F)(F)F)ccnc1C[S+]([O-])c1nc2ccccc2[nH]1,None,None,CHEMBL4495564,None,2020,None,CHEMBL480,LANSOPRAZOLE,CHEMBL480,6.41,False,http://www.openphacts.org/units/Nanomolar,3341963,=,52,True,=,None,IC50,nM,None,390.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,0.39
1,Dtt Insensitive,19964200,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,Cc1c(-c2cnccn2)ssc1=S,None,None,CHEMBL4495564,None,2020,None,CHEMBL178459,OLTIPRAZ,CHEMBL178459,6.68,False,http://www.openphacts.org/units/Nanomolar,3341991,=,52,True,=,None,IC50,nM,None,210.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,0.21
2,Dtt Insensitive,19964201,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,O=c1sn(-c2cccc3ccccc23)c(=O)n1Cc1ccccc1,None,None,CHEMBL4495564,None,2020,None,CHEMBL3545157,TIDEGLUSIB,CHEMBL3545157,7.10,False,http://www.openphacts.org/units/Nanomolar,3342067,=,52,True,=,None,IC50,nM,None,80.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,0.08
3,Dtt Insensitive,19964202,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(O)...,None,None,CHEMBL4495564,None,2020,None,CHEMBL297453,EPIGALOCATECHIN GALLATE,CHEMBL297453,5.80,False,http://www.openphacts.org/units/Nanomolar,3342156,=,52,True,=,None,IC50,nM,None,1580.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,1.58
4,Dtt Insensitive,19964203,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,O=C1C=Cc2cc(Br)ccc2C1=O,None,None,CHEMBL4495564,None,2020,None,CHEMBL4303595,None,CHEMBL4303595,7.40,False,http://www.openphacts.org/units/Nanomolar,3342307,=,52,True,=,None,IC50,nM,None,40.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Dtt Insensitive,19964311,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,C=CC(=O)c1ccc2ccccc2c1,None,None,CHEMBL4495564,None,2020,None,CHEMBL154580,None,CHEMBL154580,5.91,False,http://www.openphacts.org/units/Nanomolar,3350392,=,52,True,=,None,IC50,nM,None,1240.0,CHEMBL4523582,Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,2697049,None,None,IC50,uM,UO_0000065,None,1.24
113,Dtt Insensitive,19964312,[],CHEMBL4495583,SARS-CoV-2 3CL-Pro protease inhibition IC50 de...,F,None,None,BAO_0000190,BAO_0000019,assay format,C[n+]1c2cc(N)ccc2cc2ccc(N)cc21.[Cl-],None,None,CHEMBL4495564,None,2020,None,C

There is no missing data

## **Data pre-processing of the bioactivity**

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**.

In [45]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

### **Iterate the *molecule_chembl_id* to a list**

In [46]:
df2.molecule_chembl_id

0          CHEMBL480
1       CHEMBL178459
2      CHEMBL3545157
3       CHEMBL297453
4      CHEMBL4303595
           ...      
112     CHEMBL154580
113     CHEMBL354349
114    CHEMBL1382627
115    CHEMBL4303664
116    CHEMBL1977761
Name: molecule_chembl_id, Length: 117, dtype: object

In [47]:
mol_cid = []
for i in df2.molecule_chembl_id:
  mol_cid.append(i)


### **Iterate *canonical_smiles* to a list**

In [48]:
canonical_smiles = []
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

### **Iterate *standard_value* to a list**

In [49]:
standard_value = []
for i in df2.standard_value:
  standard_value.append(i)

### **Join everything**

In [50]:
data_list = list(zip( mol_cid, canonical_smiles, bioactivity_class, standard_value))
df3 = pd.DataFrame(data_list, columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class', 'standard_value'])

In [51]:
df3.head(3)

,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value
0,CHEMBL480,Cc1c(OCC(F)(F)F)ccnc1C[S+]([O-])c1nc2ccccc2[nH]1,active,390.0
1,CHEMBL178459,Cc1c(-c2cnccn2)ssc1=S,active,210.0
2,CHEMBL3545157,O=c1sn(-c2cccc3ccccc23)c(=O)n1Cc1ccccc1,active,80.0


Copy to the Google Drive

In [52]:
df3.to_csv('bioactivity_preprocessed_data.csv', index=False)

In [53]:
! cp bioactivity_preprocessed_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

In [54]:
! ls "/content/gdrive/My Drive/Colab Notebooks/data"

bioactivity_data.csv  bioactivity_preprocessed_data.csv
